In [1]:
from sklearn import tree
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [2]:
key_word = ['and', 'or', 'xor', 'sysobjects', 'version', 'substr', 'len', 'substring', 'exists', 'mid', 'asc',
            'inner join', 'xp_cmdshell', 'version', 'exec', 'having', 'union', 'order', 'information schema',
            'load_file', 'load data infile', 'into outfile', 'into dumpfile']
special_word = ['%20', '%28', '--', ' ', '#', '%23']


In [3]:
def keyword_count(value):
    num = 0
    for index in key_word:
        num = num + value.count(index)
    return num

In [4]:
def bigE_count(value):
    capital_len = len(re.compile(r'[A-Z]').findall(value))
    return (capital_len / len(value))  # 大写字母频率


In [5]:
def num_count(value):
    num_len = len(re.compile(r'\d').findall(value))
    return (num_len / len(value))

In [6]:
def calc_ent(value):
    data = []
    for i in value:
        data.append(i)
    x = np.array(data)
    x_value_list = set([x[i] for i in range(x.shape[0])])  # 得到数组x的元素（不包含重复元素），即x_value_list={'c', 'b', 'a'}
    ent = 0.0
    for x_value in x_value_list:
        p = float(x[x == x_value].shape[0]) / x.shape[0]  # 计算每个元素出现的概率
        logp = np.log2(p)
        ent -= p * logp
    return (ent)


In [7]:
def url_length(value):
    return (len(value))


In [8]:
def special_count(value):
    num = 0
    for index in special_word:
        num = num + value.count(index)
    return num / len(value)

In [9]:
def get_features(temp, value):
    temp.append(url_length(value))
    temp.append(calc_ent(value))
    temp.append(num_count(value))
    temp.append(bigE_count(value))
    temp.append(keyword_count(value))
    temp.append(special_count(value))
    return temp

In [10]:
value = []
label = []
temp = []
data = []
LaBel = []

with open('normal_less.csv', 'rb') as f:
    data_value = [x.decode('utf-8').strip() for x in f.readlines()]
    for i in data_value:
        value.append(i)
        label.append(0)  # 正常用户输入内容
    f.close()

with open('sqlnew.csv', 'rb') as f:
    data_value = [x.decode('utf-8').strip() for x in f.readlines()]
    for i in data_value:
        value.append(i)
        label.append(1)  # 有sql注入嫌疑的输入内容
    f.close()

print(value[:10])

for i in range(0, len(value)):
    # for i in range(0,10):
    if len(value[i]) != 0:
        temp = get_features(temp, value[i])
        data.append(temp)
        LaBel.append(label[i])
        temp = []

data = np.array(data)
LaBel = np.array(LaBel)
# 保存特征数据

print(data[:25])
print(LaBel[:25])
# test
X_train, X_test, y_train, y_test = train_test_split(data, LaBel, test_size=0.3, random_state=1)
sc = StandardScaler()
sc.fit(X_train)
# 对数据集进行处理
sql_tree = tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2,
                                       min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=6,
                                       random_state=None, max_leaf_nodes=None,
                                       min_impurity_decrease=0.0, min_impurity_split=None, presort=False)

print("***************start****************")

sql_tree_save = sql_tree.fit(X_train, y_train)
score = cross_val_score(sql_tree, data, LaBel, cv=5)
print(score)

print("**************saveing***************")
# joblib.dump(sql_tree_save, 'C:/Users/Birdman/Desktop/data/sql/sql.model')
print("Training score:%f" % (sql_tree.score(X_train, y_train)))
print("Testing score:%f" % (sql_tree.score(X_test, y_test)))
y_pred = sql_tree.predict(X_test)
print("Testing precision score:%f" % (accuracy_score(y_test, y_pred)))
print("Testing precision score:%f" % (precision_score(y_test, y_pred)))
print("Testing recall score:%f" % (recall_score(y_test, y_pred)))


['_%3D1498591621808', 'code%3Dzs_000001%2Czs_399001%2Czs_399006%26cb%3Dfortune_hq_cn%26_%3D1498591852632', '_%3D1498591951848%26list%3Dml_sh600030', '6053%26ri%3Dzb6-00f%7E-04gUry-01h-0RC%26tn%3D1%26en%3D4L9RATiSWJ84cSXYcX1d2pqRNDaJDyf4dP2hUCLUFiZhTaWU7eK5dl5slq5E7SVvDi63oAKGfcJUNiy27jClo3XeVPICvXbMtyQIUjoFYYlnKQp0XlF0M2zO7d7ub-vumAdasuEscaZLfT5w2tfIKsHlHIn49b0u1Af1JUmZeCeIVpuTgEmDmxYz9GkgA-CCk-qVMZ-V2AfzTEe7HCLcljdw1NcVt-H26P2-dz8IEEj3n9DdQGwpErgkGR6SSbaJpPQu0IgaWmPZEZu-umlvd0urhn88rdu9-Dmn4hYWD2T-menxBL9BSplyy74mnP_4DZXvZ4Ggf4n-k6WXtWPDQYekUxo6XItLiBIOTt5znJ4l9AfspnGY604PtLC0WKwyjqwq27Z4zR9JZsSXxngF9UJFi9JrCW_yldU07E3r3q9LuHrEoy8%3E%3E', 'b1498592370545%3D1', 'v%3D13111002', 'COLLCC%3D3442798258%26', 't%3Dcheck%26rec%3Dstratus%26etyp%3Dconnect%26zone%3Dzibo5_cnc%26url%3D119.188.143.32%26errCnt%3D327%26uid%3Dd0a47beafc75e1549c7fdc23530fd959%26uif%3DCNC%7CBeiJing-114.251.186.13%26tvid%3D7706069409%26defi%3D2%26dlod%3D1%26darea%3D1%26ppapi%3Dfalse%26trkip%3D119.188.143.32%26trkon%3D0%26